# Give data tags
The purpose of this notebook is to read the h5out-tagged file provided by stephen and create query the spotify api for the correct features for each of the songs.

In [12]:
# my spotify api 
from spotify_api import SpotifyAPIWrapper

# dataset manipulation
import pandas as pd
import numpy as np

In [13]:
# Read the h5out-tagged 
df = pd.read_csv("h5out-tagged.csv")
print(df.head())


                             Artist name  \
0                               Mastodon   
1                            Los Chichos   
2                             Foxy Brown   
3  Ram\xc3\xb3n Vargas;Vladimir Jurowski   
4                              Enthroned   

                                               Title  Year  Loudness  Key  \
0                                  Deep Sea Creature  2001    -3.306    5   
1                               No Quieras Marcharte  1984   -10.764    0   
2                                            If I...     0    -9.035    1   
3  Werther - Lyric Drama in four Acts/Act I/Alors...     0   -23.095    7   
4               Ad Te Clamamus Exsvles Mortva Liberi  2010   -20.359    2   

   Danceability  Energy    Tempo  isTop100  
0           0.0     0.0  173.205         0  
1           0.0     0.0  150.955         0  
2           0.0     0.0   93.056         0  
3           0.0     0.0  127.113         0  
4           0.0     0.0   90.660         0  


In [14]:
# seperate the artist names and the song names
artist_names = df.iloc[:,0]
song_names = df.iloc[:,1]
exists_in_top_100 = df.iloc[:,-1]

# check that the length of artist names and song names are the same
if (len(artist_names) == len(song_names)):
    print("Datasets match")
else:
    print("Error, differing number of songs and artists")

Datasets match


In [1]:
# loop over each one of the songs, and get the spotify song id for that song
# NOTE: currently starting with the first 10 songs to act as a test

# create a reference to my api
spotify = SpotifyAPIWrapper()

# request an api token for spotify to use
# TODO: move this to the constructor
spotify.get_spotify_api_token()
print(spotify.access_token)

#The last successful search
# the number of rows being interograted is currently 10
number_of_rows = 1335

# keeps track of where the dataset should be added to, as some errors may occur and features not able to be found on spotify
next_dataset_row = 0

# declare a response dataframe
outputframe = pd.DataFrame(columns=("Artist name", "Title", "spotify_id", "acousticness", "dancability", "energy", "instrumentalness", "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence", "isTop100"), index=np.arange(0,number_of_rows))

# for our test set of 10 songs
for i in range(0 ,number_of_rows):
    title = song_names[i]
    artist = artist_names[i]
    isTop100 = exists_in_top_100[i]

    # get the relevant song id
    songid = spotify.search_spotify_song_id(artist_name=artist, song_name=title)
    
    # check for None being returned for failed results
    if (songid != False):
        # get the features from the api
        features = spotify.get_features_from_id(songid)
        if (features != False):
            # Add the features to a new row in a new dataframe
            outputframe.loc[next_dataset_row] = [artist, title, features["id"], features["acousticness"], features["danceability"], features["energy"], features["instrumentalness"], features["key"], features["liveness"], features["loudness"], features["mode"], features["speechiness"], features["tempo"], features["valence"], isTop100]

            # increment the next row to enter to
            next_dataset_row = next_dataset_row + 1



print(outputframe)


NameError: name 'SpotifyAPIWrapper' is not defined

In [9]:
outputframe.to_csv("featured_tagged2.csv")

# Getting More positive results from the Billboard Dataset
Idea here is to take a random sample from the full billboard top 100 dataset and add more positives to our model so we can train it.


In [15]:
# Extracting hits from the dataset
import random as rand
billboardDF = pd.read_csv("hot_stuff_2.csv")

song_names_bill = billboardDF.iloc[:,3]
artist_names_bill = billboardDF.iloc[:,4]
    

In [16]:
# allocate the sample size and how many samples we are adding
SAMPLE_SIZE = 3000
DATASET_SIZE = len(song_names_bill)
collected_songs_name = []
collected_songs_artist = []

# Now take a random sample of 1000 songs, making sure that they do not match the already fetched songs.

# Collect a new random number, if it has already been picked, then pick a new one, if not then add it to the list
collected_randoms = []
def withdraw_random():
    sample_no = rand.randint(0, DATASET_SIZE)
    if (sample_no in collected_randoms):
        collected_randoms.append(sample_no)
        return withdraw_random()
    else:
        return sample_no


for i in range(0, SAMPLE_SIZE):
    sample_no = withdraw_random()
    collected_songs_name.append(song_names_bill[sample_no])
    collected_songs_artist.append(artist_names_bill[sample_no])
    
print(collected_songs_name)


 L.A.', 'The Plastic Man', 'I Found A Love Oh What A Love', 'The One', 'We Are The Young', 'Man On The Moon', 'Get At Me Dog', 'Twerk', 'Cook With Honey', "Since You've Been Gone", 'Poor Boy', "Buy U A Drank (Shawty Snappin')", 'How Do You Feel The Morning After', 'Peace Of Mind', 'Paying The Cost To Be The Boss', "Haven't Stopped Dancing Yet", 'Mine', 'Molly', "Uptight (Everything's Alright)", 'Miss Me', 'Life In Mono', 'Prisoner (Captured By Your Eyes)', 'Java', 'Never Enough', 'Wait For You', 'The Best Disco In Town', 'I Want To Hold Your Hand', 'Soul Meets Body', 'Pray For You', 'As Good As I Once Was', 'Cool Aid', 'Will You Marry Me?', 'Take Time', '99 Luftballons', 'Shake Your Bon-Bon', 'Bella y Sensual', 'Why Me', 'Smack That', 'Whodunit', 'In My Dreams', 'I Honestly Love You', 'Last Date', 'Faith', 'Holy Grail', 'The Sound Of Goodbye', 'Only The Lonely (Know How I Feel)', 'My First Night With You', "U Should've Known Better", 'Tell Me Why', 'Before The Next Teardrop Falls', 'Ju

In [20]:
# Now use that data on the spotify api
# create a reference to my api

# THIS is for use with datasets that ARE TOP 100, and so it will always have the value of 1


spotify = SpotifyAPIWrapper()
spotify.get_spotify_api_token()

# check that the songname list and artistnamelist are the same length
# if (len(songnamelist) != len(artistnamelist)):
#     print("The two lists are not of the same length")
#     return False

# keeps track of where the dataset should be added to, as some errors may occur and features not able to be found on spotify
NUMBER_OF_ROWS = len(collected_songs_name)
next_dataset_row = 0

# declare a response dataframe
outputframe_more = pd.DataFrame(columns=("Artist name", "Title", "spotify_id", "acousticness", "dancability", "energy", "instrumentalness", "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence", "isTop100"), index=np.arange(0,NUMBER_OF_ROWS))

# for our test set of 10 songs
for i in range(0 ,NUMBER_OF_ROWS):
    title = collected_songs_name[i]
    artist = collected_songs_artist[i]
    isTop100 = 1

    # get the relevant song id
    songid = spotify.search_spotify_song_id(artist_name=artist, song_name=title)
    
    # check for None being returned for failed results
    if (songid != False):
        # get the features from the api
        features = spotify.get_features_from_id(songid)
        if (features != False):
            # Add the features to a new row in a new dataframe
            outputframe_more.loc[next_dataset_row] = [artist, title, features["id"], features["acousticness"], features["danceability"], features["energy"], features["instrumentalness"], features["key"], features["liveness"], features["loudness"], features["mode"], features["speechiness"], features["tempo"], features["valence"], isTop100]

            # increment the next row to enter to
            next_dataset_row = next_dataset_row + 1


print(outputframe_more)

NameError: name 'songnamelist' is not defined

In [10]:
extraframe = getFromSpotifyAPI(collected_songs_name, collected_songs_artist)
extraframe.to_csv("extra_frames_from_top_100_2.csv")

is_url': 'https://api.spotify.com/v1/audio-analysis/2ScFhA8p6gkIfLNjzK0fun', 'duration_ms': 249613, 'time_signature': 3}
searching endpointhttps://api.spotify.com/v1/search?type=track&q=Lonestar+Mr.%20Mom
{'danceability': 0.505, 'energy': 0.842, 'key': 10, 'loudness': -3.316, 'mode': 1, 'speechiness': 0.0475, 'acousticness': 0.0576, 'instrumentalness': 0, 'liveness': 0.111, 'valence': 0.671, 'tempo': 172.136, 'type': 'audio_features', 'id': '0jSIovJtKxEXH6vI4oRx7I', 'uri': 'spotify:track:0jSIovJtKxEXH6vI4oRx7I', 'track_href': 'https://api.spotify.com/v1/tracks/0jSIovJtKxEXH6vI4oRx7I', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/0jSIovJtKxEXH6vI4oRx7I', 'duration_ms': 208693, 'time_signature': 4}
searching endpointhttps://api.spotify.com/v1/search?type=track&q=Gary%20Toms%20Empire+Drive%20My%20Car
{'danceability': 0.802, 'energy': 0.683, 'key': 0, 'loudness': -8.909, 'mode': 1, 'speechiness': 0.0425, 'acousticness': 0.056, 'instrumentalness': 2.39e-05, 'liveness': 0.0662,

TypeError: can only concatenate str (not "SSLError") to str

In [11]:
extraframe.to_csv("extra_frames_from_top_100_2.csv")

NameError: name 'extraframe' is not defined

In [1]:
# Do this again for the second lump of song names fetched in "more_data"
# In this instance to it for the first 200 songs
import pandas as pd
import numpy as np

more_data = pd.read_csv("more_data.csv")
artists = more_data["Artist Name"]
songs = more_data["Title"]

print("Dataset loaded")

Dataset loaded


In [4]:
spotify = SpotifyAPIWrapper()
spotify.get_spotify_api_token()

# Take a range of these songs
ran = range(18500, 20000)
RANGE = list(ran)

song_range = songs[RANGE]
artist_range = artists[RANGE]

print(song_range)

range_frame = pd.DataFrame(columns=("Artist name", "Title", "spotify_id", "acousticness", "dancability", "energy", "instrumentalness", "key", "liveness", "loudness", "mode", "speechiness", "tempo", "valence"), index=np.arange(0,200))

next_dataset_row = 0

for i in ran:
    title = song_range[i]
    artist = artist_range[i]

    # get the relevant song id
    songid = spotify.search_spotify_song_id(artist_name=artist, song_name=title)
    
    # check for None being returned for failed results
    if (songid != False):
        # get the features from the api
        features = spotify.get_features_from_id(songid)
        if (features != False):
            # Add the features to a new row in a new dataframe
            range_frame.loc[next_dataset_row] = [artist, title, features["id"], features["acousticness"], features["danceability"], features["energy"], features["instrumentalness"], features["key"], features["liveness"], features["loudness"], features["mode"], features["speechiness"], features["tempo"], features["valence"]]

            # increment the next row to enter to
            next_dataset_row = next_dataset_row + 1

print(range_frame)
range_frame.to_csv("range_frame_23.csv")


i.spotify.com/v1/tracks/6fPOMDQjhpriegRiBpTNK7', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6fPOMDQjhpriegRiBpTNK7', 'duration_ms': 196893, 'time_signature': 4}
searching endpointhttps://api.spotify.com/v1/search?type=track&q=Kevin%20Max+Baby_%20I%27m%20Your%20Man
{'danceability': 0.398, 'energy': 0.629, 'key': 9, 'loudness': -6.495, 'mode': 1, 'speechiness': 0.0297, 'acousticness': 0.602, 'instrumentalness': 0, 'liveness': 0.104, 'valence': 0.196, 'tempo': 176.097, 'type': 'audio_features', 'id': '43n3LlrYB1eveNSaTQuWft', 'uri': 'spotify:track:43n3LlrYB1eveNSaTQuWft', 'track_href': 'https://api.spotify.com/v1/tracks/43n3LlrYB1eveNSaTQuWft', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/43n3LlrYB1eveNSaTQuWft', 'duration_ms': 219080, 'time_signature': 4}
searching endpointhttps://api.spotify.com/v1/search?type=track&q=Little%20Willie%20John+Leave%20My%20Kitten%20Alone
{'danceability': 0.502, 'energy': 0.638, 'key': 8, 'loudness': -9.673, 'mode': 1, 'speechi

In [35]:
print(range_frame)
range_frame.to_csv("range_frame_21.csv")

                  Artist name                               Title  \
0        The Jeff Healey Band         It Could All Get Blown Away   
1                Anthony Cruz                          Make it up   
2          Thomas Battenstein              Jag Vet En Dejlig Rosa   
3                        Devo      Sloppy (I Saw My Baby Gettin')   
4    Pep Laguarda & Tapineria                   Caseta del Plater   
..                        ...                                 ...   
537        Danielle Bollinger  When the Broken Hearted Love Again   
538                    H-Town                           Sex Dance   
539                Dead To Me                        What's Wrong   
540           Mighty Diamonds                           Searching   
541                   Hatiras                         Final Flash   

                 spotify_id acousticness dancability energy instrumentalness  \
0    2YbL2OLsMCbd0B1bmcKyWx      0.00428       0.503  0.738         0.000109   
1    0NI3Hi